In [2]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
import cv2
#采用sklearn.svm.SVC，类似的有sklearn.svm.LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.decomposition import KernelPCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

In [4]:
def creat_tag_list(txtList):
    '''读取各行数据并录入对应列表'''
    tagDict = {'numStr':[],'sex':[],'age':[],'race':[],'face':[],'prop':[]}
    for line in txtList:
        if line.find('missing descriptor') == -1: # 没有缺失卷标才录入
            tagDict['numStr'].append(line[1:5])
            for keyName in tagDict.keys():
                if keyName != 'numStr':
                    list_append(tagDict,line,keyName)
    le = LabelEncoder()
    for keyName in tagDict.keys():
        if keyName in ['sex','age','race','face']:
            tagDict[keyName] = np.array(tagDict[keyName])
            # tagDict[keyName] = le.fit_transform(tagDict[keyName]) # 转换为编码
    return tagDict

def list_append(tagDict,line,tagStr):
    '''读取一个数据加入列表中'''
    idx = line.find(tagStr)
    if tagStr == 'prop': # 格式特殊，单独处理
        idxR = line.find(')',idx)
        idxL = line.find('(',idx) # 找出左括号
    else:
        idxR = line.find(')',idx)
        idxL = line.rfind(' ',0,idxR) # 空格所在的索引
    tagDict[tagStr].append(line[idxL+1:idxR])
    if tagDict[tagStr][-1] == '':
        tagDict[tagStr][-1] = 'null'

def read_tag(tagFile):
    '''读取tag文件'''
    file = open(tagFile,mode='r')
    txtList = file.readlines() # 包含各行文本的列表
    file.close()
    tagDict = creat_tag_list(txtList) # 讲对应类别信息录入各列表中
    return tagDict

def read_data(dataPath,tagDict):
    dataList = []
    for fileName in tagDict['numStr']:
        curData = read_1data(dataPath,fileName)
        if len(read_1data(dataPath,fileName)) != 128*128:
            print('图片尺寸过大，进行降采样')
            curData = curData.reshape(512,512)
            curData = cv2.pyrDown(curData,curData)
            curData = cv2.pyrDown(curData,curData)
            curData = curData.flatten()
        dataList.append(curData)
    dataList = np.vstack(dataList)
    return dataList

def read_1data(dataPath,fileName):
    '''读取tag文件'''
    curPath = dataPath+'\\'+fileName
    with open(curPath, 'rb') as f:
        content = f.read()
    # 将字节数组转成 128*128 的图像
    data = np.frombuffer(content, dtype=np.uint8)
    # data = data.reshape(128,128)
    # cv2.imshow('real', data.reshape(128,128))
    # cv2.waitKey(0)
    return data

'''读取文件、标签'''
dataPath1 = 'D:\\py\\PatternRecognition\\final\\人脸图像识别\\face\\rawdataDR'
dataPath2 = 'D:\\py\\PatternRecognition\\final\\人脸图像识别\\face\\rawdataDS'
tagFile1 = 'D:\\py\\PatternRecognition\\final\\人脸图像识别\\face\\faceDR'
tagFile2 = 'D:\\py\\PatternRecognition\\final\\人脸图像识别\\face\\faceDS'
h,w = 128,128 # 图片长宽
tagDict1 = read_tag(tagFile1)
read_1data(dataPath1,tagDict1['numStr'][0])
data1 = read_data(dataPath1,tagDict1)
print('data size',data1.size)
X1 = data1

# 加载DS数据集
tagDict2 = read_tag(tagFile2)
read_1data(dataPath2,tagDict2['numStr'][0])
data2 = read_data(dataPath2,tagDict2)
X2 = data2

图片尺寸过大，进行降采样
图片尺寸过大，进行降采样
data size 32718848


In [6]:
'''提取特征'''
X1 = data1
X2 = data2
'''LLE'''
n_components = 300
lle = LocallyLinearEmbedding(n_components=n_components, n_neighbors=100, random_state=42)
X1_reduced = lle.fit_transform(X1)
X1 = X1_reduced
print('X1 shape',X1.shape)

X2_reduced = lle.fit_transform(X2)
X2 = X2_reduced
print('X2 shape',X2.shape)

X1 shape (1997, 300)
X2 shape (1996, 300)


In [7]:
test_size = 0.3

In [10]:
'''二分类，各种分类器'''
clf_knn = KNeighborsClassifier()
clf_rf = RandomForestClassifier()
# clf_SVC = SVC(kernel='linear')
clf_SVC = SVC(kernel='rbf', probability=True)
clf_gNB = GaussianNB()
clf_bNB = BernoulliNB()
clf_sgd = SGDClassifier(random_state=42)

'''单个数据集交叉验证'''
keyName = 'sex'
# 划分数据集和测试集
# X_train, X_test, y_train, y_test = train_test_split\
#     (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
X_train, X_test, y_train, y_test = train_test_split\
    (X1, tagDict1[keyName], test_size=test_size, random_state=0)
# X_train, X_test, y_train, y_test = X1,X2,tagDict1[keyName],tagDict2[keyName]
for clf, label in \
    zip([clf_knn,clf_rf,clf_SVC,clf_gNB,clf_bNB,clf_sgd], \
        ['knn','rf','SVC','g Naive Bayes','b Naive Bayes','sgd']):
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    clf.fit(X_train, y_train)
    accuracy_train = clf.score(X_train, y_train)
    accuracy_test = clf.score(X_test, y_test)
    print("AUC-cross:%0.2f(+/- %0.2f) train:%0.2f test:%0.2f [%s]" % (scores.mean(), scores.std(), accuracy_train, accuracy_test, label))
    # print(classification_report(y_test, clf.predict(X_test)))

AUC-cross:0.78(+/- 0.02) train:0.85 test:0.75 [knn]
AUC-cross:0.82(+/- 0.02) train:1.00 test:0.75 [rf]
AUC-cross:0.85(+/- 0.02) train:0.97 test:0.79 [SVC]
AUC-cross:0.72(+/- 0.03) train:0.72 test:0.66 [g Naive Bayes]
AUC-cross:0.73(+/- 0.05) train:0.80 test:0.69 [b Naive Bayes]
AUC-cross:0.84(+/- 0.03) train:0.86 test:0.77 [sgd]


In [18]:
'''二分类'''
keyName = 'sex'
X_train, X_test, y_train, y_test = train_test_split\
    (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
# X_train, X_test, y_train, y_test = X1,X2,tagDict1[keyName],tagDict2[keyName]
'''stacking'''
lr = LogisticRegression()
# RFC = RandomForestClassifier(n_estimators=100
#     , min_impurity_decrease=0.0025
#     , random_state= 420, n_jobs=8)
RFC = RandomForestClassifier()
f_esti = lr
stacking = StackingClassifier(estimators=\
    [('KNN',clf_knn), ('rf',clf_rf), ('SVC',clf_SVC), ('gNB',clf_gNB), ('bNB',clf_bNB), ('sgd',clf_sgd)],  final_estimator=f_esti)
scores = cross_val_score(stacking, X_train, y_train, cv=5, scoring='roc_auc')
stacking.fit(X_train,y_train)
accuracy_train = stacking.score(X_train, y_train)
accuracy_test = stacking.score(X_test, y_test)
print("AUC-cross:%0.2f(+/- %0.2f) train:%0.2f test:%0.2f [%s]" % (scores.mean(), scores.std(), accuracy_train, accuracy_test, 'stacking'))
print(classification_report(y_test, stacking.predict(X_test)))

AUC-cross:0.90(+/- 0.01) train:1.00 test:0.84 [stacking]
              precision    recall  f1-score   support

      female       0.81      0.77      0.79       473
        male       0.86      0.88      0.87       725

    accuracy                           0.84      1198
   macro avg       0.83      0.83      0.83      1198
weighted avg       0.84      0.84      0.84      1198



In [ ]:
'''SVC调参'''
keyName = 'sex'
X_train, X_test, y_train, y_test = train_test_split\
    (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
scoresList = []
for k in ['linear', 'poly', 'rbf', 'sigmoid']:
    # clf = SVC(kernel=k, probability=True)
    clf = SVC(kernel=k)
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
    clf.fit(X_train,y_train)
    accuracy_train = clf.score(X_train, y_train)
    accuracy_test = clf.score(X_test, y_test)
    print("AUC-cross:%0.2f(+/- %0.2f) train:%0.2f test:%0.2f [%s]" % (scores.mean(), scores.std(), accuracy_train, accuracy_test, k))

AUC-cross:0.73(+/- 0.02) train:0.65 test:0.62 [linear]
AUC-cross:0.86(+/- 0.01) train:0.78 test:0.71 [poly]
AUC-cross:0.85(+/- 0.01) train:0.91 test:0.80 [rbf]
AUC-cross:0.64(+/- 0.02) train:0.64 test:0.62 [sigmoid]


In [9]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
import warnings
warnings.filterwarnings("ignore")
'''多分类，各种分类器'''
clf_knn = KNeighborsClassifier()
clf_rf = RandomForestClassifier()
# clf_SVC = SVC(kernel='linear')
clf_SVC = SVC(kernel='sigmoid', probability=True)
clf_gNB = GaussianNB()
clf_bNB = BernoulliNB()
clf_sgd = SGDClassifier(random_state=42)

'''指定数据、目标值'''
keyName = 'age'
X_train, X_test, y_train, y_test = train_test_split\
    (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
# X_train, X_test, y_train, y_test = X1,X2,tagDict1[keyName],tagDict2[keyName]

for clf, label in \
    zip([clf_knn,clf_rf,clf_SVC,clf_gNB,clf_bNB,clf_sgd], \
        ['knn','rf','SVC','g Naive Bayes','b Naive Bayes','sgd']):
    
    if clf in [clf_SVC]:
        clf = OneVsRestClassifier(clf)
        # clf = OneVsOneClassifier(clf)
        # clf = OutputCodeClassifier(clf, code_size=2, random_state=0)
    clf.fit(X_train, y_train)
    accuracy_train = clf.score(X_train, y_train)
    accuracy_test = clf.score(X_test, y_test)
    print("train:%0.2f test:%0.2f [%s]" % (accuracy_train, accuracy_test, label))
    # print('       ',label)
    # print(classification_report(y_test, clf.predict(X_test)))

train:0.88 test:0.82 [knn]
train:1.00 test:0.81 [rf]
train:0.79 test:0.78 [SVC]
train:0.67 test:0.64 [g Naive Bayes]
train:0.85 test:0.78 [b Naive Bayes]
train:0.82 test:0.79 [sgd]


In [19]:
'''多分类'''
import warnings
warnings.filterwarnings("ignore")
'''指定数据、目标值'''
keyName = 'age'
X_train, X_test, y_train, y_test = train_test_split\
    (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
# X_train, X_test, y_train, y_test = X1,X2,tagDict1[keyName],tagDict2[keyName]
'''stacking'''
lr = LogisticRegression()
RFC = RandomForestClassifier()
f_esti = RFC
stacking = StackingClassifier(estimators=[('KNN',clf_knn), ('SVC',clf_SVC)],  final_estimator=f_esti)
stacking.fit(X_train,y_train)
accuracy_train = stacking.score(X_train,y_train)
accuracy_test = stacking.score(X_test,y_test)
print("train:%0.2f test:%0.2f [%s]" % (accuracy_train, accuracy_test, 'stacking'))
print(classification_report(y_test, stacking.predict(X_test)))

train:0.98 test:0.85 [stacking]
              precision    recall  f1-score   support

       adult       0.89      0.96      0.92       943
       child       0.59      0.50      0.54        96
      senior       0.89      0.48      0.62        52
        teen       0.57      0.40      0.47       107

    accuracy                           0.85      1198
   macro avg       0.73      0.59      0.64      1198
weighted avg       0.84      0.85      0.84      1198



In [ ]:
'''LLE调参'''
nCo_a,nCo_b,nCo_step = 20,60,10
nNei_a,nNei_b,nNei_step = 5,55,10

rNum = int((nCo_b-nCo_a)/nCo_step)
cNum = int((nNei_b-nNei_a)/nNei_step)
scoresDict = {'avg':np.zeros([rNum,cNum]),'knn':np.zeros([rNum,cNum]),'rf':np.zeros([rNum,cNum]),\
    'SVC':np.zeros([rNum,cNum]),'g Naive Bayes':np.zeros([rNum,cNum]),'b Naive Bayes':np.zeros([rNum,cNum]),\
            'sgd':np.zeros([rNum,cNum])}

i = 0
for n_components in range(nCo_a,nCo_b,nCo_step):
    j = 0
    for n_neighbors in range(nNei_a,nNei_b,nNei_step):
        X1 = data1
        X2 = data2
        '''提取特征'''
        '''LLE'''
        lle = LocallyLinearEmbedding(n_components=n_components, n_neighbors=n_neighbors, random_state=42)
        X1_reduced = lle.fit_transform(X1)
        X1 = X1_reduced
        X2_reduced = lle.fit_transform(X2)
        X2 = X2_reduced

        '''二分类，各种分类器'''
        clf_knn = KNeighborsClassifier()
        clf_rf = RandomForestClassifier(n_estimators=30)
        # clf_SVC = SVC(kernel='linear')
        clf_SVC = SVC(kernel='rbf', probability=True)
        clf_gNB = GaussianNB()
        clf_bNB = BernoulliNB()
        clf_sgd = SGDClassifier(random_state=42)

        '''单个数据集交叉验证'''
        keyName = 'sex'
        # 划分数据集和测试集
        X_train, X_test, y_train, y_test = train_test_split\
            (np.vstack((X1,X2)), np.hstack((tagDict1[keyName],tagDict2[keyName])), test_size=test_size, random_state=0)
        # X_train, X_test, y_train, y_test = X1,X2,tagDict1[keyName],tagDict2[keyName]
        scoresList = [] # 记录一次的平均AUC
        for clf, label in \
            zip([clf_knn,clf_rf,clf_SVC,clf_gNB,clf_bNB,clf_sgd], \
                ['knn','rf','SVC','g Naive Bayes','b Naive Bayes','sgd']):
            scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc')
            clf.fit(X_train, y_train)
            accuracy_train = clf.score(X_train, y_train)
            accuracy_test = clf.score(X_test, y_test)
            scoresList.append(scores.mean())
            scoresDict[label][i,j] = scores.mean()
            # print("AUC-cross:%0.2f(+/- %0.2f) train:%0.2f test:%0.2f [%s]" % (scores.mean(), scores.std(), accuracy_train, accuracy_test, label))
            # print(classification_report(y_test, clf.predict(X_test)))
        scoresDict['avg'][i,j] = np.mean(scoresList)
        print('=======>n_components,n_neighbors',n_components,n_neighbors)
        for label in ['avg','knn','rf','SVC','g Naive Bayes','b Naive Bayes','sgd']:
            print(label,scoresDict[label][i,j])
        j += 1
    i += 1